# 01 — Data Quality & Cleaning Pipeline 

This notebook loads the NovaCred credit application JSON dataset, performs a data quality audit, and demonstrates remediation steps in code.

**Input:** `data/raw/raw_credit_applications.json` (kept local / not committed due to PII fields)  
**Outputs:**  
- `data/processed/applications_clean.csv` (cleaned, may still contain PII)  
- `data/processed/applications_analysis_ready.csv` (PII removed for analysis)

**Quality checks covered (with counts and %):** duplicate records, missing/incomplete values, inconsistent data types, inconsistent categorical coding, inconsistent date formats, invalid/impossible values, and internal consistency checks (accuracy).

In [4]:
# Imports
import json
import warnings
from pathlib import Path

import numpy as np
import pandas as pd

## 0. Load & schema sanity check

Goal: confirm the dataset loads correctly, has the expected shape, and contains the key columns required for downstream quality checks.

- **Input file:** `data/raw/raw_credit_applications.json`
- **Method:** load JSON array → flatten nested objects using `pd.json_normalize(sep=".")`
- **Sanity checks:** print dataset shape, preview 3 rows, and verify presence of key expected columns (schema check).

In [5]:
from pathlib import Path
import json
import pandas as pd

# Raw file lives in data/raw/ (no repo)
data_path = Path("../data/raw/raw_credit_applications.json")

# Read JSON
with data_path.open("r", encoding="utf-8") as f:
    apps = json.load(f)

# Flatten nested JSON into a tabular DataFrame
df_raw = pd.json_normalize(apps, sep=".")

# Basic sanity checks
print("rows, cols:", df_raw.shape)
display(df_raw.head(3))

# Check expected columns
expected = {
    "_id",
    "spending_behavior",
    "processing_timestamp",
    "applicant_info.full_name",
    "applicant_info.ssn",
    "financials.annual_income",
    "financials.credit_history_months",
    "financials.debt_to_income",
    "financials.savings_balance",
    "decision.loan_approved",
}

missing = expected - set(df_raw.columns)
print("Missing expected columns:", missing)

rows, cols: (502, 21)


,_id,spending_behavior,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,...,financials.credit_history_months,financials.debt_to_income,financials.savings_balance,decision.loan_approved,decision.rejection_reason,loan_purpose,decision.interest_rate,decision.approved_amount,financials.annual_salary,notes
0,app_200,"[{'category': 'Shopping', 'amount': 480}, {'ca...",2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,...,23,0.20,31212,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
1,app_037,"[{'category': 'Rent', 'amount': 608}, {'catego...",NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,...,51,0.18,17915,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
2,app_215,"[{'category': 'Rent', 'amount': 109}]",NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,...,41,0.21,37909,True,NaN,vacation,3.7,59000.0,NaN,NaN


Missing expected columns: set()


Dataset loaded successfully (502 rows, 21 columns) and all key expected columns are present.

## 1. `spending_behavior` (array of objects) — Data quality checks

`spending_behavior` is a nested array (list of `{category, amount}` objects).  
We validate completeness, type consistency, validity (e.g., negative amounts), formatting consistency (category casing), and within-application duplicates.

### 1A. Create an analysis table

`spending_behavior` is stored as an array of objects per application.  
To run data quality checks at the entry level, we **explode** the array so each `(category, amount)` becomes one row in a new table (`spend`).

In [6]:
# Build an exploded table: one row per spending entry
spend = (
    df_raw[["_id", "spending_behavior"]]
    .dropna(subset=["spending_behavior"])
    .explode("spending_behavior", ignore_index=True)
)

# Extract fields safely
spend["category"] = spend["spending_behavior"].map(lambda x: x.get("category") if isinstance(x, dict) else None)
spend["amount"] = pd.to_numeric(
    spend["spending_behavior"].map(lambda x: x.get("amount") if isinstance(x, dict) else None),
    errors="coerce"
)

print("Total spending entries:", len(spend))
display(spend[["_id", "category", "amount"]].head(10))

Total spending entries: 827


,_id,category,amount
0,app_200,Shopping,480
1,app_200,Rent,790
2,app_200,Alcohol,247
3,app_037,Rent,608
4,app_037,Dining,96
5,app_037,Healthcare,243
6,app_215,Rent,109
7,app_024,Fitness,575
8,app_184,Entertainment,463
9,app_275,Entertainment,571


### 1B. Missing / incomplete

We check completeness at two levels:
1) application-level: is `spending_behavior` missing for any applications?
2) entry-level: after exploding the array, are any entries missing `category` or `amount`?

In [7]:
# Total number of rows in the main table
n_apps = len(df_raw)

# Count applications where spending_behavior is missing (NaN)
missing_sb_apps = df_raw["spending_behavior"].isna().sum()
print("Applications missing spending_behavior:", missing_sb_apps, f"({missing_sb_apps/n_apps*100:.2f}%)")

# Total number of spending entries (rows) after exploding the array
n_entries = len(spend)

# Count entries where category is missing
missing_cat = spend["category"].isna().sum()

# Count entries where amount is missing
missing_amt = spend["amount"].isna().sum()

print("Missing category entries:", missing_cat, f"({missing_cat/max(n_entries,1)*100:.2f}%)")
print("Missing amount entries:", missing_amt, f"({missing_amt/max(n_entries,1)*100:.2f}%)")

Applications missing spending_behavior: 0 (0.00%)
Missing category entries: 0 (0.00%)
Missing amount entries: 0 (0.00%)


### 1C. Inconsistent data types

Validate that each spending entry has:
- `category` as text (string)
- `amount` as a numeric value 

In [8]:
# Count values that aren't Python strings
non_string_cat = (~spend["category"].dropna().map(lambda x: isinstance(x, str))).sum()
print("Non-string category entries:", non_string_cat,
      f"({non_string_cat/max(n_entries, 1)*100:.2f}%)")

# Count amounts that are missing or not numeric (NaN after pd.to_numeric)
non_numeric_amount = spend["amount"].isna().sum()
print("Non-numeric/missing amount entries:", non_numeric_amount,
      f"({non_numeric_amount/max(n_entries, 1)*100:.2f}%)")


Non-string category entries: 0 (0.00%)
Non-numeric/missing amount entries: 0 (0.00%)


### 1D. Invalid values
Check for impossible values such as negative spending amounts.

In [9]:
# Count negative amounts (should be 0)
neg_amount = int((spend["amount"] < 0).sum())

print("Negative amount entries:", neg_amount, f"({neg_amount/max(n_entries,1)*100:.2f}%)")

Negative amount entries: 0 (0.00%)


### 1E. Category formatting consistency
Compare raw vs normalized category values (lowercase + trimmed) to detect casing/spacing inconsistencies.

In [10]:
cat_raw = spend["category"].astype("string")
cat_norm = cat_raw.str.strip().str.lower()

# Count unique categories before vs after normalization
n_raw = cat_raw.dropna().nunique()
n_norm = cat_norm.dropna().nunique()

print("Distinct raw categories:", n_raw)
print("Distinct normalized categories:", n_norm)

# Show normalized categories that have multiple raw variants (case/spacing differences)
variants = (
    pd.DataFrame({"raw": cat_raw, "norm": cat_norm})
    .dropna()
    .drop_duplicates()
    .groupby("norm")["raw"]
    .unique()
)

variants = variants[variants.map(len) > 1]

if len(variants) == 0:
    print("No formatting variants found (case/spacing is consistent).")
else:
    display(variants.head(20))

Distinct raw categories: 15
Distinct normalized categories: 15
No formatting variants found (case/spacing is consistent).


## 2. Main table — Data quality checks

### 2A. Duplicate records (Uniqueness)

We check for duplicate records using key identifiers:
- duplicate application IDs (`_id`)
- duplicate applicant identifiers (`applicant_info.ssn`, excluding missing values)

We report counts and percentages, and later apply a remediation rule during the cleaning step.

In [11]:
# Total number of applications
n = len(df_raw)

# 1) Duplicate application IDs (_id)
# _id should uniquely identify an application
dup_id = df_raw["_id"].duplicated().sum()

# 2) SSN duplicates (excluding missing)
# SSN identifies an applicant, so we check duplicates only among non-missing SSNs
ssn = df_raw["applicant_info.ssn"]
missing_ssn = ssn.isna().sum()
dup_ssn_excl_missing = ssn.dropna().duplicated().sum()
n_non_missing_ssn = ssn.notna().sum()

print("Duplicate _id:", dup_id, f"({dup_id/n*100:.2f}%)")
print("Missing SSN:", missing_ssn, f"({missing_ssn/n*100:.2f}%)")
print("Duplicate SSN (excluding missing):", dup_ssn_excl_missing,
      f"({dup_ssn_excl_missing/max(n_non_missing_ssn,1)*100:.2f}% of non-missing SSNs)")

Duplicate _id: 2 (0.40%)
Missing SSN: 5 (1.00%)
Duplicate SSN (excluding missing): 3 (0.60% of non-missing SSNs)


Results show 2 duplicate application IDs (0.40%), and 3 duplicate SSNs among non-missing values (0.60%). These indicate uniqueness issues and will be addressed in the cleaning pipeline.

In [12]:
# Show duplicated _id rows
dup_id_rows = df_raw[df_raw["_id"].duplicated(keep=False)].sort_values("_id")
display(dup_id_rows[["_id", "applicant_info.full_name", "applicant_info.ssn", "processing_timestamp"]])

# Show duplicated SSN rows (excluding missing SSNs)
dup_ssn_rows = df_raw[df_raw["applicant_info.ssn"].notna() & df_raw["applicant_info.ssn"].duplicated(keep=False)] \
    .sort_values("applicant_info.ssn")
display(dup_ssn_rows[["_id", "applicant_info.full_name", "applicant_info.ssn"]].head(20))

,_id,applicant_info.full_name,applicant_info.ssn,processing_timestamp
383,app_001,Stephanie Nguyen,427-90-1892,NaN
455,app_001,Stephanie Nguyen,NaN,NaN
8,app_042,Joseph Lopez,652-70-5530,NaN
354,app_042,Joseph Lopez,652-70-5530,NaN


,_id,applicant_info.full_name,applicant_info.ssn
8,app_042,Joseph Lopez,652-70-5530
354,app_042,Joseph Lopez,652-70-5530
92,app_088,Susan Martinez,780-24-9300
122,app_016,Gary Wilson,780-24-9300
16,app_101,Sandra Smith,937-72-8731
499,app_234,Samuel Hill,937-72-8731


These tables show example duplicate cases:
- `_id` duplicates (e.g., `app_001`) appear as repeated application records.
- SSN duplicates appear either within the same `_id` (e.g., `app_042`) or across different `_id`s (e.g., the same SSN linked to multiple applications).
These will be handled during remediation.

### 2B. Missing / incomplete records (Completeness)

We quantify missingness for each field (count and %) to identify incomplete columns and prioritize remediation.

In [13]:
# Count missing (NaN) values per column
missing_count = df_raw.isna().sum().sort_values(ascending=False)

# Percent missing per column
missing_pct = (df_raw.isna().mean() * 100).round(2).sort_values(ascending=False)

# Combine into one table for reporting
missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_%": missing_pct
})

display(missing_table)

,missing_count,missing_%
notes,500,99.60
financials.annual_salary,497,99.00
loan_purpose,452,90.04
processing_timestamp,440,87.65
decision.rejection_reason,292,58.17
decision.approved_amount,210,41.83
decision.interest_rate,210,41.83
financials.annual_income,5,1.00
applicant_info.ip_address,5,1.00
applicant_info.ssn,5,1.00


Results show that the most incomplete fields are `notes` (99.6%), `financials.annual_salary` (99.0%), `loan_purpose` (90.0%), and `processing_timestamp` (87.7%).  
Decision-related fields are conditionally missing: `decision.approved_amount` and `decision.interest_rate` are missing for ~41.8% of applications, and `decision.rejection_reason` is missing for ~58.2%, consistent with approval/denial outcomes.

By looking into the table we can also notice that the dataset contains both `financials.annual_income` and `financials.annual_salary`.  
`annual_salary` is almost always missing, suggesting inconsistent field usage. This can cause confusion in downstream analysis.
We address this in remediation by merging `annual_salary` into `annual_income` when `annual_income` is missing, then dropping `annual_salary`.

### 2C. Inconsistent data types (Consistency)

We verify that numeric fields are consistently stored as numeric values (not strings).  
This is important before any modeling or aggregation.

In [14]:
numeric_cols = [
    "financials.annual_income",
    "financials.annual_salary",
    "financials.credit_history_months",
    "financials.debt_to_income",
    "financials.savings_balance",
    "decision.interest_rate",
    "decision.approved_amount",
]

# For each numeric column, count how many values are int/float/str (excluding NaN)
type_counts = {}
for col in numeric_cols:
    s = df_raw[col]
    counts = s.dropna().map(lambda x: type(x).__name__).value_counts()
    type_counts[col] = counts.to_dict()

# Convert the dictionary into a table
type_counts_df = pd.DataFrame(type_counts).fillna(0).astype(int).T

print("Type counts by column (non-missing values):")
display(type_counts_df)

# Percentages (out of non-missing values in each column)
non_missing = df_raw[numeric_cols].notna().sum()
type_pct_df = (type_counts_df.div(non_missing, axis=0) * 100).round(2)

print("Type percentages by column (non-missing values):")
display(type_pct_df)

Type counts by column (non-missing values):


,int,str,float
financials.annual_income,488,8,1
financials.annual_salary,0,0,5
financials.credit_history_months,502,0,0
financials.debt_to_income,0,0,502
financials.savings_balance,502,0,0
decision.interest_rate,0,0,292
decision.approved_amount,0,0,292


Type percentages by column (non-missing values):


,int,str,float
financials.annual_income,98.19,1.61,0.2
financials.annual_salary,0.00,0.00,100.0
financials.credit_history_months,100.00,0.00,0.0
financials.debt_to_income,0.00,0.00,100.0
financials.savings_balance,100.00,0.00,0.0
decision.interest_rate,0.00,0.00,100.0
decision.approved_amount,0.00,0.00,100.0


**Findings (type consistency):**
- `financials.annual_income` has mixed types (mostly integers, but some values are stored as strings), indicating inconsistent typing across records.
- `financials.annual_salary` is rarely populated and appears separately from `annual_income`, suggesting inconsistent schema usage for income.
- Other numeric fields (`credit_history_months`, `debt_to_income`, `savings_balance`, and decision terms) are consistently numeric when present.

As a quick schema validation step, we verify that fields expected to be strings contain only string values (for non-missing entries), and that `decision.loan_approved` behaves as a boolean. This helps catch unexpected structures (e.g., numbers/lists in text fields) before remediation.

In [15]:
expected_str = [
    "_id",
    "applicant_info.full_name",
    "applicant_info.email",
    "applicant_info.ssn",
    "applicant_info.ip_address",
    "applicant_info.gender",
    "applicant_info.date_of_birth",
    "applicant_info.zip_code",
    "decision.rejection_reason",
    "loan_purpose",
    "notes",
]

# For these, check if any non-missing values are NOT strings
non_string_counts = {}
for col in expected_str:
    if col in df_raw.columns:
        s = df_raw[col]
        non_string_counts[col] = int(s.dropna().map(lambda x: not isinstance(x, str)).sum())

display(pd.Series(non_string_counts).sort_values(ascending=False))

# Boolean check for loan_approved
import numpy as np

vals = df_raw["decision.loan_approved"].dropna().unique()
print("Unique loan_approved values:", vals)
print("Types:", [type(v) for v in vals])
print("All boolean (Python bool or numpy.bool_)?", all(isinstance(v, (bool, np.bool_)) for v in vals))

_id                             0
applicant_info.full_name        0
applicant_info.email            0
applicant_info.ssn              0
applicant_info.ip_address       0
applicant_info.gender           0
applicant_info.date_of_birth    0
applicant_info.zip_code         0
decision.rejection_reason       0
loan_purpose                    0
notes                           0
dtype: int64

Unique loan_approved values: [False  True]
Types: [<class 'numpy.bool'>, <class 'numpy.bool'>]
All boolean (Python bool or numpy.bool_)? True


**Result:** All fields expected to be strings contain only string values for non-missing entries (0 non-string values found in each).  
`decision.loan_approved` contains only boolean values (`True`/`False`, stored as `numpy.bool_`), so it is type-consistent with the schema.